# Biostat 257 Homework 2

**Due May 1 @ 11:59PM**

In [ ]:
versioninfo()

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. \
**Solution:** The mean and variance of the response vector is:

From givin information we know, $\bar {Y} = X\beta$ and $Var(Y) = \sigma^2I_n + Z\Sigma Z^T$, and thus we can calculate the log likelihood function as:
\begin{equation}
\begin{split}
l & = -\frac{n}{2}\ln(2\pi)-\frac{1}{2}\ln( |Var(Y_i)|) - \frac{1}{2}(Y-\mu)^TVar(Y_i)^{-1}(Y-\mu)\\
& = -\frac{n}{2}\ln(2\pi)-\frac{1}{2}\ln (|\sigma^2I_n + Z\Sigma Z^T|) - \frac{1}{2}(Y-X\beta)^T(\sigma^2I_n + Z\Sigma Z^T)^{-1}(Y-X\beta)
\end{split}
\end{equation}

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [1]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    res        :: Vector{T}
    storage_q  :: Vector{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    res        = similar(y)
    storage_q  = Vector{T}(undef, size(Z, 2))
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, res, storage_q, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [24]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    

    # TODO: compute and return the log-likelihood
    rmul!(obs.Z,LowerTriangular(L))
#     Σ = σ²*I + obs.Z*transpose(obs.Z);
    Σ = LinearAlgebra.BLAS.syrk!('U','N', 1.0, obs.Z, 1.0, σ²*Matrix{Float64}(I, n, n))
    Σchol = cholesky!(Symmetric(Σ));
    μin = obs.y-obs.X*β
    l = -(n//2)*log(2*pi) - (1//2)*logdet(Σchol) -  (1//2) * dot(μin, Σchol\μin)#transpose(μ)*inv(Var)*μ 
    return l
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [26]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3;

# predictors
X  = [ones(n) randn(n, p - 1)];
Z  = [ones(n) randn(n, q - 1)];
# parameter values
β  = [2.0; -1.0; rand(p - 2)];
σ² = 1.5;
Σ  = fill(0.1, q, q) + 0.9I;
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n);

# form an LmmObs object
obs = LmmObs(y, X, Z);

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [7]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I

# From istribution pacakage:
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3247.262248197789

In [23]:
L = Matrix(cholesky(Σ).L)
Z  = [ones(n) randn(n, q - 1)];
LinearAlgebra.BLAS.syrk!('U','N', 1.0, Z*L, 1.0, σ²*Matrix{Float64}(I, n, n))

2000×2000 Array{Float64,2}:
 3.90635  0.195727  -0.1906    2.66111   …  0.810258   0.863338    2.02967
 0.0      3.00061    1.29769   0.538609     1.09562    1.36188     0.59826
 0.0      0.0        4.73628  -2.28328      1.0485    -0.146293   -0.638801
 0.0      0.0        0.0       7.21102      0.88839    2.63182     3.32422
 0.0      0.0        0.0       0.0          1.02721    3.49556     3.26985
 0.0      0.0        0.0       0.0       …  0.894418   0.909553    1.50334
 0.0      0.0        0.0       0.0          0.773802   1.36463     2.79639
 0.0      0.0        0.0       0.0          0.644937  -0.0519489   2.21156
 0.0      0.0        0.0       0.0          1.03901    0.565792    0.161774
 0.0      0.0        0.0       0.0          0.901547   2.09993     2.68483
 0.0      0.0        0.0       0.0       …  0.893249   1.76382     2.3941
 0.0      0.0        0.0       0.0          0.861202  -0.886445   -0.126038
 0.0      0.0        0.0       0.0          0.866513   0.521893    1.2

Check that your answer matches that from Distributions.jl

In [27]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3247.4568580638256

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertError`.

In [293]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

In [149]:
using Profile

Profile.clear()
@profile logl!(obs, β, L, σ²)
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
     6         0 In[143]                    9 logl!(::LmmObs{Float64}, ::Arra...
    10         0 In[143]                   10 logl!(::LmmObs{Float64}, ::Arra...
     2         0 In[143]                   12 logl!(::LmmObs{Float64}, ::Arra...
     5         5 @Base\array.jl           352 copy
    19         1 @Base\boot.jl            331 eval
    19         0 @Base\essentials.jl      712 #invokelatest#1
    19         0 @Base\essentials.jl      711 invokelatest
     1         0 @Base\operators.jl       529 *
    19         0 @Base\task.jl            358 (::IJulia.var"#15#18")()
    19         0 ...ia\src\eventloop.jl     8 eventloop(::ZMQ.Socket)
    19         0 ...\execute_request.jl    67 execute_request(::ZMQ.Socket, :...
    19         0 ...\SoftGlobalScope.jl   218 softscope_include_string(::Modu...
     5         0 ...rc\LinearAlgebra.jl   347 copy_oftype
     1         

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [8]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 
  memory estimate:  30.55 MiB
  allocs estimate:  5
  --------------
  minimum time:     15.825 ms (0.00% GC)
  median time:      16.269 ms (0.00% GC)
  mean time:        18.826 ms (13.78% GC)
  maximum time:     27.073 ms (35.44% GC)
  --------------
  samples:          266
  evals/sample:     1

In [28]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 
  memory estimate:  61.08 MiB
  allocs estimate:  11
  --------------
  minimum time:     64.936 ms (0.00% GC)
  median time:      71.102 ms (7.36% GC)
  mean time:        71.121 ms (6.99% GC)
  maximum time:     80.559 ms (12.83% GC)
  --------------
  samples:          71
  evals/sample:     1

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [83]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

0.007720162732469542

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [ ]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.